In [1]:
import gamspy as gp
import gamspy.math as gpm
from gamspy import Sum, Card

import sys
import numpy as np
import pandas as pd

# Stochastic production problem

In [3]:
m = gp.Container()
t = gp.Set(m,'t',records=['Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']) 
d = gp.Parameter(m,'d',domain=t,records=np.array([4, 8, 20, 12, 6, 2]))
# Scale to thousands
d[t] = 1000 * d[t]
initialProd = gp.Parameter(m,'initialProd',records=4000,description='production in June')
initialInv = gp.Parameter(m,'initialInv',records=2000,description='inventory at end of June')

increaseProd = 1.50 # cost per unit to increase production
decreaseProd = 1.00 # cost per unit to decrease production
lowInv = 8000  # maximum (low) inventory level
lowHoldCost = 0.2 # Inventory holding cost (low)
highHoldCost = 0.5 # Inventory holding cost (high)
shortageCost = 1.0 # Shortage cost

ell = gp.Variable(m,'ell','positive',domain=t,description="low inventory level")
h = gp.Variable(m,'h','positive',domain=t,description="high inventory level")
u = gp.Variable(m,'u','positive',domain=t,description="increase in production")
n = gp.Variable(m,'n','positive',domain=t,description="decrease in production")
r = gp.Variable(m,'r','positive',domain=t,description="shortage")
y = gp.Variable(m,'y','positive',domain=t,description="demand met")
p = gp.Variable(m,'p','positive',domain=t,description="production")
i = gp.Variable(m,'i','positive',domain=t,description="inventory")

split_inv_eq = gp.Equation(m,'split_inv_eq',domain=t) 
split_inv_eq[t] = i[t] == ell[t] + h[t]

prod_change_eq = gp.Equation(m,'prod_change_eq',domain=t) 
prod_change_eq[t] = p[t] - p[t-1] - initialProd.where[t.first] == u[t] - n[t]

inv_bal_eq = gp.Equation(m,'inv_bal_eq',domain=t) 
inv_bal_eq[t] = i[t-1] + initialInv.where[t.first] + p[t] == y[t] + i[t]

shortage_eq = gp.Equation(m,'shortage_eq',domain=t) 
shortage_eq[t] = r[t] >= d[t] - y[t]

sp = gp.Model(m,'sp',
    equations=m.getEquations(),
    problem=gp.Problem.LP,
    sense=gp.Sense.MIN,
    objective=Sum(t, lowHoldCost * ell[t] + highHoldCost * h[t] +
                 increaseProd * u[t] + decreaseProd * n[t] +
                 shortageCost * r[t]),
)

ell.up[t] = lowInv
y.up[t] = d[t]

sp.solve(options=gp.Options(qcp="cplex"),output=None)

TypeError: Domain item must be type Set, Alias, ImplicitSet or str but found `<class 'gamspy._algebra.expression.Expression'>`

In [ ]:
display(p.l.records)

In [ ]:
m = gp.Container()

# PUT YOUR STOCHASTIC MODEL HERE
# You may want to copy the original data statement and perhaps
# equations and then edit them to add the scenario information

sp.solve(options=gp.Options(qcp="cplex"),output=None)

In [ ]:
display(p.l.records)